# Wave function postprocesing

Here we wish to do postprocesing on Quantum Espresso data extracted using
`pw_export.x` after a band calculation using `ascii=.true.` option. See
https://www.quantum-espresso.org/Doc/INPUT_pw_export.html for more info.

## Checking Orthonormality for Wavefunctions(Fourrier coefficients)

Assume we have our data files, first thing we wish to do is check if wavefunctions are orthonormal. For this we need a procedure that will read from our output the following things: number of points on k-grid and number of wavefunctions (these are actually Fourrier coefficients $C_{nk}^G$ where $n$ is a band index, $k$ is a point in k-space and $G$ is the lattice vector in reciprodal (Fourrier) space).

First, let's make functions that can gather our data for $C_{nk}^G$, ${\bf G}$-vectors, ${\bf k}$-path and one-electron functions eigenvalues $E_{n{\bf{k}}}$: 

In [ ]:
import numpy as np
import re as re

def Gather_C(path,k_point,details=False):    
    FILE = open (path+'/wfc.{}'.format(k_point),"r")
    nbnd=0
    igwx=0
    WF=[]
    if details:
        print("Now performing for k-point={}:\n".format(k_point))
    for line in FILE:
        if 'Info' in line:
            found_nbnd = re.search('nbnd="(.+?)"',line)
            if found_nbnd:
                nbnd = int(found_nbnd.group(1))
        if 'igwx' in line:
            found_igwx=re.search('igwx="(.+?)"',line)
            if found_igwx:
                igwx = int(found_igwx.group(1))
        for n in range(nbnd):
            wf_n=[]
            if '<Wfc.{} type='.format(n+1) in line:
                for i in range(igwx):
                    nextLine=next(FILE)
                    A = (nextLine.replace("\n","")).split(",")
                    C = complex(float(A[0]),float(A[1]))

                    wf_n.append(C)
                WF.append(wf_n)
    if details:
        print("\tReading wave-fuctions: DONE")
    WF=np.array(WF)
    return WF

def Gather_G(path,k_point,details=False):
    FILE = open (path+'/grid.{}'.format(k_point),"r")
    size=0
    GV=[]
    if details:
        print("Gathering G-vectors for k-point={}:\n".format(k_point))
    for line in FILE:
        if '<grid type=' in line:
            size = re.search('size="(.+?)"',line)
            if size:
                size = int(int(size.group(1))/3)
            for i in range(size):
                nextLine=next(FILE)
                g = (nextLine.replace("\n","")).split()
                G_i = np.array([float(g[0]),float(g[1]),float(g[2])])
                GV.append(G_i)
    if details:
        print("\tReading G-vectors: DONE")
    GV=np.array(GV)
    return GV

def Gather_K(path,details=False):
    FILE = open (path+'/index.xml',"r")
    KV=[]
    if details:
        print("Gathering K-vectors...\n")
    for line in FILE:
        if '<Kmesh' in line:
            found_nk = re.search('nk="(.+?)"',line)
            if found_nk:
                nk = int(found_nk.group(1))
        if '<k type=' in line:
            for k in range(nk):
                nextLine=next(FILE)
                k = nextLine.split()
                k_i = [float(k[0]),float(k[1]),float(k[2])]
                KV.append(k_i)
    if details:
        print("\tGathering K-vectors: DONE")
    KV=np.array(KV)
    return KV

def Gather_E(path,details=False,units='Rydberg'):
    FILE = open (path+'/index.xml',"r")
    nbnd=0
    nk=0
    E=[]
    if details:
        print("Now performing for k-point={}:\n".format(k_point))
    for line in FILE:
        if '<Eigenvalues' in line:
            found_nk = re.search('nk="(.+?)"',line)
            if found_nk:
                nk = int(found_nk.group(1))
            found_nbnd = re.search('nbnd="(.+?)"',line)
            if found_nbnd:
                nbnd = int(found_nbnd.group(1))
        for k in range(1,nk+1):
            E_k=[]
            if '<e.{} type='.format(k) in line:
                for n in range(nbnd):
                    nextLine=next(FILE)
                    #print(nextLine.split())
                    #print(nextLine)
                    e = float(nextLine)
                    E_k.append(e)
                E.append(E_k)
    if details:
        print("\tReading wave-fuctions: DONE")
    E=np.array(E)
    if units=='Hartree':
        return E*0.5
    if units=='Electronvolt':
        return E*13.605698066
    if units=='Rydberg':
        return E

Now that we know how to gather data, we can define a function `Check_Orthonormality` that can check if our $C_{nk}^G$ coefficients are indeed orthonormial:

In [ ]:
def Check_Orthonormality(WF, tol=1e-13,details=False):
    nbnd = len(WF)
    Ort_MAT=np.zeros((nbnd,nbnd),dtype=complex)
    for i in range(nbnd):
        for j in range(i,nbnd):
            A=np.array(WF[i])
            B=np.array(WF[j])
            Ort_MAT[i][j]=np.vdot(A,B)
            Ort_MAT[j][i]=Ort_MAT[i][j]
    if details:
        print("\tVector product is: DONE")
    
    orthonormal=True
    tol_global=tol
    index = [0,0]
    for i in range(nbnd):
        if abs(abs(Ort_MAT[i][i])-1.0)<tol:
            pass
        else:
            if details:
                print ("\n\tMatrix element [{},{}] is:{}".format(i,i,Ort_MAT[i][i]))
            orthonormal=False
            ort_new=False
            tol_new=tol
            while ort_new==False:
                    tol_new=tol_new*10
                    if tol_global<tol_new: tol_global=tol_new; index=[i,i]
                    if abs(abs(Ort_MAT[i][i])-1.0)<tol_new:
                        if details:
                            print("\tTolerance for {}".format(tol_new))
                        ort_new=True
    for i in range(nbnd):
        for j in range(i+1,nbnd):
            if abs(Ort_MAT[i][j])<tol:
                pass
            else:
                if details:
                    print ("\n\tMatrix element: {},{} is:{}".format(i,j,abs(Ort_MAT[i][j])))
                orthonormal=False
                ort_new=False
                tol_new=tol
                while ort_new==False:
                    tol_new=tol_new*10
                    if tol_global<tol_new: tol_global=tol_new; index=[i,j]
                    if abs(Ort_MAT[i][j])<tol_new:
                        if details: 
                            print("\tTolerance for {}".format(tol_new))
                        ort_new=True                    
    print('\nOrthonormality {} for tolerance {}'.format(orthonormal,tol))
    if not orthonormal:
        print('Orthonormal for tolerance {} in index {}\n'.format(tol_global,index))

We can test these functions for $n_{bands} = 20, 40, 60, 80, 100, 120, 140$:

In [ ]:
for nbnd in ['20', '40' , '60' , '80', '100' , '120' , '140']:
    path = "/home/mjocic/data_paradox/nbnd_"+nbnd
    tolerance=1e-13
    print('\n\t****FOR N_bands = '+nbnd+' ******')
    for k in range(1,42):
        print('\nk-point number {}'.format(k))
        C = Gather_C(path,k)
        Check_Orthonormality(C,tolerance)

## Calculating the $k \cdot p$ Hamiltonian

### Obtaining the $k\cdot p$ Matrix Elements

Now that we have collected our $C_{nk}^G$ coefficients we can proceed to find matrix elements of $\langle m |{\bf k}\cdot {\bf p} | n \rangle$ where
$m$ and $n$ represent the band index and ${\bf k}$ and ${\bf p}$ represent a certain k-point in Fourrier space and momentum operator respectively. Starting from
$$
    \int d^3 {\bf r} u_{mk}^* {\bf k} \cdot {\bf p} u_{nk}
$$
we arrive to the expression:
$$
    i {\bf k} \cdot \sum_G (C_{mk}^G)^* {\bf G} C_{nk}^G
$$
which will need to calculate. We can see that we have one part of the expression but we need to incorporate $G$-vectors as well. This is done by reading the `grid.*` files and extracting data considering G-points. The procedure will be similar to the one for orthonormality check, except we need to insert ${\bf G}$-vectors that we obtained from `grid.*` file.

Eigenproblem that we're trying to solve is:
$$
H_{\bf k}({\bf r})u_{n{\bf k}}({\bf r})=E_{n {\bf k}} u_{n{\bf k}}({\bf r})
$$
Where we can expand functions $u_{n{\bf k}}$ around a certain k-point, since Bloch functions form an orhonormial basis, which corresponds to some eigenvector with eigenvalue $E_{n {\bf k}}$. For our case, let that be
the R symmetry point ${\bf k}_R$ so that we can write:
$$
u_{n{\bf k}}({\bf r}) = \sum_n C_{nk} u_{nR}({\bf r})
$$
and use it to obtain:
$$
\sum_n \left[ E_{nR} + 
    \frac{\hbar^2}{2 m_0}(k^2-k_R^2) \right] C_{nR} \delta_{mn}
    +\sum_n \frac{\hbar}{m_0}
\left\langle u_{mR}\left|({\bf k}-{\bf k}_R)\cdot {\bf p } \right|u_{nR}\right\rangle C_{nk} = \sum_n E_{n\bf{k}} C_{nk} \delta_{mn}
.
$$
This can be represented in matrix form where $h_{mn}$ are matrix coefficients of Hamiltonian which we now need do diagonalize:
$$
h_{mn}=\left[E_{nR} + \frac{\hbar^2}{2 m_0}(k^2-k_R^2) \right] \delta_{mn}
+\sum_n \frac{\hbar}{m_0}
\left\langle u_{mR}\left|({\bf k}-{\bf k}_R)\cdot {\bf p } 
\right| u_{nR}\right\rangle .
$$
We focus now on obtaining the Hamiltonian matrix $h_{mn}$, then diagonalizing it and obtaining the eigenvalues.

In [ ]:
import numpy as np
import re as re

def Compute_Matrix_Element(G_vec,C_terms,m,n):
    A = np.array(C_terms[m])
    B = np.array(C_terms[n])
    Gx=G_vec[:,0]
    Gy=G_vec[:,1]
    Gz=G_vec[:,2]
    G = [np.vdot(A,Gx*B),np.vdot(A,Gy*B),np.vdot(A,Gz*B)]
    return np.array(G)  

def Compute_H(k_mesh,k_point,E_k,
              G_space,C_terms,a,
              units='Rydberg',
              details=False):
    uni=['Hartree',1.0]
    if units=='Rydberg':
        uni=['Rydberg',2.0]
    if units=='Electronvolt':
        uni=['Electronvolt',27.211396132]
    tpiba = 2*np.pi/a
    tpiba2=tpiba**2
    kR2= np.dot(k_point,k_point)
    H=[]
    for k_i in range(len(k_mesh)):
        k2 = np.dot(k_mesh[k_i],k_mesh[k_i])
        kkR = k_mesh[k_i]-k_point
        H_i = np.zeros((len(C_terms),len(C_terms)),dtype=complex)
        for m in range(len(C_terms)):
            for n in range(len(C_terms)):
                CGC = Compute_Matrix_Element(G_space,C_terms,m,n)
                KdP = np.dot(kkR,CGC)
                if m==n:
                    H_i[m][m] = E_k[m] + 0.5*tpiba2*(k2-kR2) + tpiba2*KdP
                else:
                    H_i[m][n] = tpiba2*KdP
        H_i = (np.linalg.eigvalsh(H_i))
        H.append(H_i)
        if details:
            print("eigenvalues of H_{} in units of {}".format(k_i+1,uni[0]))
            print((H_i)*uni[1])
    print('Calculating bands: DONE')
    return np.array(H)*uni[1]

In [66]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_R=31
E=Gather_E(path,units='Hartree')
E_R = E[k_R-1]
k_points= Gather_K(path)
k_Rpoint = k_points[k_R-1]
G_R = Gather_G(path,k_R)
C_R = Gather_C(path,k_R)

In [67]:
H = Compute_H(k_points,k_Rpoint,E_R,
              G_R,C_R,a=10.8,
              units='Hartree',
              details=True)

eigenvalues of H_1 in units of Hartree
[-0.52551125 -0.51178279 -0.51178111 -0.4599347  -0.45993411 -0.44209486
 -0.40540771 -0.40540749 -0.137349   -0.05291033 -0.05290908 -0.04336978
  0.02372056  0.02693716  0.02693747  0.03215368  0.03215429  0.041471
  0.1866382   0.22633338  0.22633344  0.26283449  0.26283458  0.26314
  0.31889142  0.34825202  0.34825232  0.37323324  0.37323368  0.39026449
  0.42746777  0.42746778  0.47737633  0.50091156  0.52964851  0.52964891
  0.57634411  0.58025403  0.59230218  0.5923028   0.61177511  0.61486426
  0.61487167  0.61725629  0.61725813  0.64139168  0.65774846  0.66268446
  0.66268681  0.66592245  0.70857615  0.70857749  0.73473018  0.7347529
  0.73630311  0.76412171  0.78289673  0.80922291  0.80922346  0.81972555
  0.81973876  0.83981467  0.88450618  0.88450686  0.88532232  0.9012378
  0.91555448  0.91555525  0.9307527   0.935344    0.93534658  0.98237242
  0.98238219  1.03252159  1.04124037  1.04124056  1.07000155  1.10119974
  1.10121037  1.165

eigenvalues of H_7 in units of Hartree
[-0.52297166 -0.5145403  -0.51317702 -0.49269194 -0.48660538 -0.4393412
 -0.42551964 -0.42165975 -0.19881105 -0.05404423 -0.01743254 -0.01245323
  0.00529926  0.01760518  0.0186096   0.02191852  0.03375947  0.05436097
  0.22137398  0.23231254  0.23951227  0.25438329  0.27288839  0.29588455
  0.30571088  0.31239551  0.31783658  0.35567105  0.355866    0.37155912
  0.40493774  0.41529018  0.42884863  0.43451295  0.49044168  0.50335972
  0.51000018  0.51590991  0.5245864   0.53422591  0.57879794  0.59206747
  0.59804421  0.6079865   0.62263886  0.64700461  0.65043591  0.6567848
  0.66655245  0.69206057  0.71476479  0.72025579  0.72276808  0.73091763
  0.74332674  0.75044091  0.75354264  0.77231335  0.78484832  0.8044961
  0.80852416  0.8148724   0.83683393  0.84559935  0.85151907  0.86804816
  0.88417197  0.91563202  0.91711172  0.91853649  0.92526953  0.94266622
  0.96287561  0.96397274  0.97077054  0.99013118  1.00028159  1.00739937
  1.02325825  1

eigenvalues of H_13 in units of Hartree
[-5.21358851e-01 -5.15042737e-01 -5.14914477e-01 -4.97283886e-01
 -4.90079663e-01 -4.33914980e-01 -4.30852184e-01 -4.27501325e-01
 -2.11851043e-01 -5.20922952e-02 -1.73822855e-02  3.32178814e-04
  3.35940966e-03  1.43398548e-02  1.51927040e-02  2.11179000e-02
  3.96838442e-02  8.15101530e-02  1.89310797e-01  1.98695459e-01
  2.56593040e-01  2.63998600e-01  2.74414475e-01  2.92827690e-01
  2.99113168e-01  3.11366676e-01  3.46018818e-01  3.61952001e-01
  3.62660126e-01  3.74500956e-01  3.88160225e-01  3.99932132e-01
  4.34739721e-01  4.49901265e-01  4.68428196e-01  4.75169136e-01
  4.92258269e-01  5.03109649e-01  5.49901439e-01  5.51536040e-01
  5.69679583e-01  5.73754025e-01  5.97740854e-01  6.13799397e-01
  6.14765810e-01  6.20359260e-01  6.38032407e-01  6.40860998e-01
  6.60503489e-01  6.63158779e-01  6.81501045e-01  6.94582627e-01
  7.05502420e-01  7.37187862e-01  7.51348513e-01  7.53711836e-01
  7.71715923e-01  7.83831531e-01  7.97626710e-01  

eigenvalues of H_19 in units of Hartree
[-0.52503553 -0.5148005  -0.51091683 -0.4865644  -0.48162466 -0.43676576
 -0.42789558 -0.41623386 -0.18707949 -0.05375182 -0.04046567 -0.00678913
  0.00956556  0.01522859  0.0224035   0.02277512  0.02817763  0.06217509
  0.21317961  0.22373384  0.25193981  0.25398397  0.28106879  0.28647422
  0.30655883  0.33564746  0.33911874  0.3533955   0.36498527  0.3703671
  0.37539178  0.42153928  0.43014038  0.44717215  0.45413043  0.49526294
  0.50338609  0.52066208  0.53808864  0.56743652  0.57869052  0.58465377
  0.61626503  0.62381733  0.63359149  0.65937962  0.66599603  0.67631104
  0.68156604  0.68543608  0.69217161  0.70694349  0.71189314  0.7356383
  0.7536153   0.76327883  0.77589197  0.77779539  0.79584103  0.80127794
  0.80994269  0.83146189  0.83579336  0.84452915  0.85535122  0.86326151
  0.88063968  0.8922041   0.91403384  0.9201046   0.92458162  0.94898943
  0.96297122  0.97965547  1.00051315  1.0168881   1.03834312  1.04555272
  1.0458538  

eigenvalues of H_25 in units of Hartree
[-0.51939653 -0.51624855 -0.51223162 -0.50279522 -0.49937267 -0.43845144
 -0.43240332 -0.42554978 -0.21685572 -0.03172403 -0.02032696 -0.01103152
 -0.00394706  0.01169607  0.02203287  0.02721241  0.03011339  0.06697936
  0.18580238  0.22585034  0.24567469  0.26602604  0.26656423  0.28344195
  0.29898755  0.31938171  0.33156252  0.36518385  0.37120533  0.37465864
  0.37593172  0.41140641  0.42943767  0.44726022  0.45791589  0.50791107
  0.50970069  0.52049172  0.52488706  0.53894886  0.55344057  0.565189
  0.58650312  0.6015039   0.61160549  0.62405489  0.64207016  0.67728713
  0.68430838  0.6896397   0.69527891  0.70550804  0.71195343  0.7197567
  0.73264936  0.73647383  0.74601691  0.7755335   0.77707128  0.78058639
  0.79925682  0.80709143  0.81964434  0.83108358  0.85418449  0.86630303
  0.87449975  0.87546825  0.88353021  0.91304974  0.93794158  0.94633256
  0.94750555  0.97325872  0.99408344  0.99816297  0.99918347  1.0097912
  1.01648963  1

eigenvalues of H_31 in units of Hartree
[-0.51624095 -0.51624095 -0.51284294 -0.51284294 -0.51284294 -0.43587512
 -0.43587512 -0.43587512 -0.24954797 -0.01710393 -0.01710393 -0.01710393
  0.01858249  0.01858249  0.04007262  0.04007262  0.04007262  0.11328372
  0.15077892  0.15077892  0.15077892  0.25742477  0.28738189  0.28738189
  0.28738189  0.34155672  0.34155672  0.3921896   0.3921896   0.3921896
  0.40800612  0.40800612  0.40800612  0.48066832  0.49342657  0.49342657
  0.49342657  0.52559971  0.52559971  0.52559971  0.60279846  0.60279846
  0.60279846  0.60642232  0.61072593  0.61072593  0.61091124  0.61091124
  0.61091124  0.63055045  0.63055045  0.63055045  0.6508267   0.6508267
  0.6508267   0.67334713  0.67334713  0.79092661  0.80773959  0.80773959
  0.80773959  0.8361378   0.88139403  0.88139403  0.88139403  0.93175144
  0.93175144  0.93175144  0.94342396  0.94342396  0.94342396  0.95830985
  0.95830985  0.95830985  0.96707248  0.96707248  1.00227866  1.01769054
  1.01769054 

eigenvalues of H_37 in units of Hartree
[-0.51839807 -0.5139219  -0.51392166 -0.50969228 -0.50480194 -0.43421034
 -0.43414034 -0.43414033 -0.23298026 -0.02308192 -0.01797732 -0.00669001
 -0.00669     0.01938369  0.02458531  0.0245854   0.04025366  0.09004235
  0.16909383  0.16909383  0.22978658  0.27750173  0.27864778  0.27864779
  0.29823361  0.30014274  0.35128005  0.38136159  0.38136163  0.38528274
  0.39458978  0.39458993  0.42989624  0.44932962  0.47861758  0.47861775
  0.51938526  0.52108953  0.52916533  0.55186194  0.55186197  0.57467928
  0.5816696   0.58322006  0.5832201   0.60599424  0.60599427  0.61244938
  0.62332544  0.68476488  0.70987386  0.7098739   0.71097782  0.73062942
  0.73243991  0.73244018  0.74186733  0.77184576  0.77184577  0.77204125
  0.78245126  0.81040796  0.81040801  0.82089789  0.8374181   0.89700353
  0.89700358  0.93820325  0.94193221  0.94193241  0.9509734   0.96110353
  0.96111774  0.97185832  0.97828146  0.98022448  1.00671869  1.00858867
  1.0093437

 Here, we insert a small code for printing our results to files
 `plotband_kp.dat` and `plotband_dft.dat` for ${\bf k}\cdot {\bf p}$ and DFT calculation, respectively:

In [92]:
# Prepare results for ploting
k2=[0.0]
d=0
for i in range(len(k_points)-1):
    d1 = k_points[i+1]-k_points[i]
    d2 = np.sqrt(np.vdot(d1,d1))
    d += d2
    k2.append(d)
k2=np.array(k2)
x=[];y_kp=[];y_dft=[]
FILE_kp = open( path+'/plotband_kp.dat','w')
FILE_dft = open( path+'/plotband_dft.dat', 'w')
for n in range(H.shape[1]):
    for k_i in range(len(k2)):
        x.append(k2[k_i])
        y_kp.append(H[k_i][n].real);
        y_dft.append(E[k_i][n])
        FILE_kp.write("{} {}\n".format (k2[k_i],H[k_i][n].real))
        FILE_dft.write("{} {}\n".format (k2[k_i],E[k_i][n].real))
    FILE_kp.write("\n")
    FILE_dft.write("\n")
FILE_kp.close()
FILE_dft.close()

Using matplotlib backend: Qt5Agg


In [ ]:
# Plot the results
import matplotlib.pyplot as plt
%matplotlib
plt.plot(x,y_kp,label='$k \cdot p$')
plt.plot(x,y_dft, label='dft')
plt.xlim(k2[0],k2[-1])
plt.ylim(-0.1,0.3)
plt.xticks([k2[0],k2[10],k2[20],k2[30],k2[40]],('$\Gamma$','M','X','R','M'))
plt.ylabel('E(k) [Hartree]')
plt.legend()
plt.show()

Having seen that we indeed have orthonormality for our one-electron functions from DFT calculation, we can move forward to checking energy levels in certain ${\bf k}$-points and their degeneracies.

We need to see what kind of symmetries are observed at a given ${\bf k}$-point. To do this we first need to obtain from our numerical calculations how many times is a certain band degenerated at that point and then find on which reducible representations does the certain point decompose.

Suppose that we have an $n$-degenerate state for a certain ${\bf k}$
that corresponds to wave-functions $\psi_{1{\bf k}},\psi_{2{\bf k}},\psi_{3{\bf k}} ,\ldots \psi_{n{\bf k}}$ and that they're symmetrical under certain symmetry operation $R$ which is represented by operator $\hat{P}_R$, we can write for some wave-function $\psi_{i{\bf k}}$ then:
$$
 \hat{P}_R \psi_{i{\bf k}}({\bf r}) = \sum_j D_{ij}(R)\psi_{i{\bf k}}({\bf r})
                           = \sum_j D_{ij}(R)e^{i{\bf k}\cdot {\bf r}}
                           u_{{\bf k}j}({\bf r})
$$
and at the same time
$$
\hat{P}_R \psi_{i{\bf k}}({\bf r}) 
                           = e^{i{\bf k}\cdot {\bf R^{-1}r}}
                           u_{{\bf k}j}(R^{-1}{\bf r})
                           =e^{i{\bf k}\cdot {\bf r}}
                           e^{i{\bf G}_R\cdot {\bf r}}
                           u_{{\bf k}j}({\bf r})
$$
where we used ${\bf k}\cdot {\bf R^{-1}r}={\bf Rk}\cdot {\bf r}$ and that for high symmetry points $R{\bf k} = {\bf k} + {\bf G}_R$ where ${\bf G}_R$ is a reciprocial lattice vector. If we multiply both equations with $u_{{\bf k}n}^*({\bf r})$ on the left and integrate over ${\bf r}$, we arrive at:
$$
\sum_{{\bf G}} (C_{{\bf k}n}^{R{\bf G}-{\bf G}_R})^* C_{{\bf k}i}
= \sum_n D_{in}(R)
$$
We can use the formula for charachter of an representation $\chi(R) = \sum_n D_{nn}(R)$ to arrive at:
$$
 \chi(R)=\sum_n\sum_{{\bf G}} (C_{{\bf k}n}^{R({\bf G}+{\bf k})-{\bf k}})^* C_{{\bf k}n}
$$
This is the formula which we will use to obtain charachters of representation that degenerate wave-functions transform to.

### Obtaining Charachters in Python

Before we proceed to use the formula above, it is useful to make a sidenote here, i.e. to show some of the classes and functions we will use in the following analysis of our results.

First class we see is the `Symmetry` class and function `Gather_Symmetries` that creates a list of instances of that class. Functions is built to read symmetries from the `index.xml` output file given by Quantum Espresso. 

In [497]:
# Create Symmetry class and Gather_Symmetries function:
import re as re

class Symmetry:
    def __init__(self, matrix, operation):
        self.matrix = matrix
        self.operation = operation
        self.char = np.trace(matrix)
    def __str__(self):
        return '{} {}\n'.format(self.matrix,self.operation)
    def __repr__(self):
        return '{} {}\n'.format(self.matrix,self.operation)

class SymmClass:
    def __init__(self, charachter, operation):
        self.charachter =charachter
        self.operation =operation
    def __str__(self):
        return '{} {}\n'.format(self.charachter,self.operation)
    def __repr__(self):
        return '{} {}\n'.format(self.charachter,self.operation) 
        
def Gather_Symmetries(path,file_name="index.xml"):
    full_path=path+"/"+file_name
    FILE = open (full_path,"r")
    n_sym=0
    inv_sym = False
    sym_list=[]
    for line in FILE:
        if '<symmops' in line:
            n_sym = re.search('nsym="(.+?)"',line)
            if n_sym:
                n_sym = int(n_sym.group(1))
    for i in range(1,n_sym+1):
        FILE = open (full_path,"r")
        for line in FILE:
            if '<info.{} name'.format(i) in line:
                sym_name = re.search('name="(.+?)"',line)
                sym_name = str(sym_name.group(1))
                nextLine=next(FILE)
                mat_type = re.search('type="(.+?)"',nextLine)
                nextLine=next(FILE)
                r = nextLine.split()
                R1 = [int(r[0]),int(r[1]),int(r[2])]
                nextLine=next(FILE)
                r = nextLine.split()
                R2 = [int(r[0]),int(r[1]),int(r[2])]
                nextLine=next(FILE)
                r = nextLine.split()
                R3 = [int(r[0]),int(r[1]),int(r[2])]
                mat = np.array([R1,R2,R3])
                R = Symmetry(mat,sym_name)
                sym_list.append(R)
    return sym_list

We can test this function as seen in the example below:

In [656]:
path = "/home/mjocic/data_paradox/nbnd_120"

symmetry_list = Gather_Symmetries(path)

#def Reduce_to_Class(symmetry_list):
for i in range(len(symmetry_list)):
    print("No. {}. Name:".format(i+1),symmetry_list[i].operation,
         '\n', symmetry_list[i].matrix, "Charachter = {}\n".format(symmetry_list[i].char))

No. 1. Name: identity 
 [[1 0 0]
 [0 1 0]
 [0 0 1]] Charachter = 3

No. 2. Name: 180 deg rotation - cart. axis [0,0,1] 
 [[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]] Charachter = -1

No. 3. Name: 180 deg rotation - cart. axis [0,1,0] 
 [[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]] Charachter = -1

No. 4. Name: 180 deg rotation - cart. axis [1,0,0] 
 [[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]] Charachter = -1

No. 5. Name: 180 deg rotation - cart. axis [1,1,0] 
 [[ 0  1  0]
 [ 1  0  0]
 [ 0  0 -1]] Charachter = -1

No. 6. Name: 180 deg rotation - cart. axis [1,-1,0] 
 [[ 0 -1  0]
 [-1  0  0]
 [ 0  0 -1]] Charachter = -1

No. 7. Name:  90 deg rotation - cart. axis [0,0,-1] 
 [[ 0  1  0]
 [-1  0  0]
 [ 0  0  1]] Charachter = 1

No. 8. Name:  90 deg rotation - cart. axis [0,0,1] 
 [[ 0 -1  0]
 [ 1  0  0]
 [ 0  0  1]] Charachter = 1

No. 9. Name: 180 deg rotation - cart. axis [1,0,1] 
 [[ 0  0  1]
 [ 0 -1  0]
 [ 1  0  0]] Charachter = -1

No. 10. Name: 180 deg rotation - cart. axis [-1,0,1] 
 [[ 0  0 -1]
 [ 0 -1  

In [216]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_R=31
#E=Gather_E(path,units='Hartree')
#E_G = E[k_G-1]
k_points= Gather_K(path)
k_Rpoint = k_points[k_R-1]
G_R = Gather_G(path,k_R)
C_R = Gather_C(path,k_R)
symmetry_list = Gather_Symmetries(path,"index.xml")

We also wish to see which wave-functions are degenerate at which points and how many times (2 or 3 times degenerate). For that we made fucntion `Gather_Degeneracies` that takes list of sorted eigenvalues and gives back a list of elements (lists) that contain the eigenvalue, degeneracy number and index (or range of indices) of the eigenvalues in the input list.

Notice that for `functions=True` function returns, alog with said standard output, a list of indices of degenerate eigenvalues that can be used in further analysis.

In [658]:
def Gather_Degeneracies(E_list,functions=False):
    deg_e=[]
    deg_u=[]
    index=0
    while index+3<E_list.shape[0]:
        if abs(E_list[index]-E_list[index+1])<1e-4:
            index += 1
            if abs(E_list[index]-E_list[index+1])<1e-4:
                index += 1
                deg_e.append([E_list[index],3,'index: {}-{}'.format(index-2,index)])
                deg_u.append([index-2,index-1,index])
            else:
                deg_e.append([E_list[index],2,'index: {}-{}'.format(index-1,index)])
                deg_u.append([index-1,index])
        else:
            deg_e.append([E_list[index],1,'index: {}'.format(index)])
            deg_u.append([index])
        index +=1
    if functions==False:
        return np.array(deg_e)
    else:
        return (deg_u),np.array(deg_e)

Let's test now this function and see if our results for ${\bf R}$-point in ${\bf k}$-space correspond to the calculated eigenvalues in our perturbation formula (we perturb around ${\bf R}$-point so the perturbation should be zero here):

In [659]:
deg_u,deg_e = Gather_Degeneracies(E_R,functions=True)
deg_h = Gather_Degeneracies(H[k_R-1])
for i in range(len(deg_e)):
    print(deg_e[i],deg_h[i])
print(deg_e.shape)
print(deg_u)

['-0.5162409537790605' '2' 'index: 0-1'] ['-0.5162409537790605' '2' 'index: 0-1']
['-0.5128429440547455' '3' 'index: 2-4'] ['-0.5128429440547455' '3' 'index: 2-4']
['-0.4358751247862201' '3' 'index: 5-7'] ['-0.4358751247862201' '3' 'index: 5-7']
['-0.2495479660081982' '1' 'index: 8'] ['-0.2495479660081982' '1' 'index: 8']
['-0.01710393499880564' '3' 'index: 9-11'] ['-0.01710393499880564' '3' 'index: 9-11']
['0.01858249019899866' '2' 'index: 12-13'] ['0.01858249019899866' '2' 'index: 12-13']
['0.04007262111570476' '3' 'index: 14-16'] ['0.04007262111570476' '3' 'index: 14-16']
['0.1132837207780535' '1' 'index: 17'] ['0.1132837207780535' '1' 'index: 17']
['0.1507789201773012' '3' 'index: 18-20'] ['0.1507789201773012' '3' 'index: 18-20']
['0.2574247740956344' '1' 'index: 21'] ['0.2574247740956344' '1' 'index: 21']
['0.2873818870259632' '3' 'index: 22-24'] ['0.2873818870259632' '3' 'index: 22-24']
['0.3415567225604785' '2' 'index: 25-26'] ['0.3415567225604785' '2' 'index: 25-26']
['0.392189

Next we wich to se what are the charachters for irreducible representation (irrep) that certain eigenvectors (wave-functions), that correspond to degenerate eigenvalues (energies), provide the basis for. In other words, by which irreducilbe reperesentation do these eigenvectors (wave-functions) transform. Of course, we need to know which symmetry our crystal structure obeys (what point/space group does it belong to), and check which irrep does that symmetry have. This can be checked on Bilbao Server's website on this adress (for point groups): http://www.cryst.ehu.es/cgi-bin/cryst/programs/representations_point.pl?tipogrupo=spg

In [245]:
def Gather_Charachters(k_start,k_stop,symmetry_list,G_R,C_R,details=False):
    charachters=[]
    dim_G = 2*abs(max(np.amax(G_R),np.amin(G_R),key=abs))+1
    if details: print('Dimension of 3x3 square matrix is:', dim_G)
    M=np.zeros((dim_G,dim_G,dim_G),dtype=int)
    if details: print('Creating G-matrix')
    for i in range(len(G_R)):
        n1,n2,n3 = G_R[i,0],G_R[i,1],G_R[i,2]
    #    print(n1,n2,n3, i)
        M[n1,n2,n3] = i
    if details: print('G-matrix: Done')

    for s in [0, 1, 6, 4, 16, 24, 25, 30, 28, 40]:
        sym_name=symmetry_list[s].operation
        sym_matrix=symmetry_list[s].matrix
        if details: print('\nNow performing for '+ sym_name)
        char=0
        for k in range(k_start,k_stop+1):
            RC_R = np.zeros(len(G_R),dtype=complex)
            for i in range(len(G_R)):
                g_i = np.matmul(sym_matrix,G_R[i]+k_Rpoint)-k_Rpoint
                n1,n2,n3=int(g_i[0]),int(g_i[1]),int(g_i[2])
                index = M[n1,n2,n3]
                RC_R[i] = C_R[k,index]
            char += np.vdot(RC_R,C_R[k])
        if details: print('charachter = ',char)
        charachters.append([int(round(char.real)), sym_name])
    if details: print('Done')
    return np.array(charachters)

Let us now check all the degeneracies and their charachters of their representations:

In [248]:
Deg_char=[]
for uk in deg_u:
    kstart=uk[0]
    kstop=uk[-1]
    char=Gather_Charachters(kstart,kstop,symmetry_list,G_R,C_R)
    Deg_char.append(np.array(char[:,0],dtype=int))

In [250]:
for i in range(len(Deg_char)):
    print(deg_u[i], Deg_char[i])

[0, 1] [ 2  2  0  0 -1 -2 -2  0  0  1]
[2, 3, 4] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[5, 6, 7] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[8] [ 1  1 -1 -1  1 -1 -1  1  1 -1]
[9, 10, 11] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[12, 13] [ 2  2  0  0 -1 -2 -2  0  0  1]
[14, 15, 16] [ 3 -1 -1  1  0 -3  1  1 -1  0]
[17] [ 1  1 -1 -1  1 -1 -1  1  1 -1]
[18, 19, 20] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[21] [1 1 1 1 1 1 1 1 1 1]
[22, 23, 24] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[25, 26] [ 2  2  0  0 -1  2  2  0  0 -1]
[27, 28, 29] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[30, 31, 32] [ 3 -1  1 -1  0  3 -1  1 -1  0]
[33] [ 1  1 -1 -1  1 -1 -1  1  1 -1]
[34, 35, 36] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[37, 38, 39] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[40, 41, 42] [ 3 -1  1 -1  0  3 -1  1 -1  0]
[43] [1 1 1 1 1 1 1 1 1 1]
[44, 45] [ 2  2  0  0 -1  2  2  0  0 -1]
[46, 47, 48] [ 3 -1 -1  1  0 -3  1  1 -1  0]
[49, 50, 51] [ 3 -1  1 -1  0 -3  1 -1  1  0]
[52, 53, 54] [ 3 -1 -1  1  0  3 -1 -1  1  0]
[55, 56] [ 2  2  0  0 -1 -2 -2  0  0  1]
[57] [ 1  

Now we can write down arrays that represent charachters for irrep of $O_h$ group (also called $m\overline{3}m$). Also we define function `char_coef` that takes charachters of two representations, one of smaller order and other of greather order and a list of numbers of elements in their class, respectively. Function returns how many times a given representation of smaller order is contained in the other representation of greather order.

In [6]:
def char_coef(g_ired,g_red,Nk):
    prod=0
    for i in range(len(Nk)):
        prod+=g_ired[i]*g_red[i]*Nk[i]
    return prod

In [8]:
char_g1 = np.array([1,1,1,1,1,1,1,1,1,1])
char_g2 = np.array([1,1,-1,-1,1,1,1,-1,-1,1])
char_g12= np.array([2,2,0,0,-1,2,2,0,0,-1])
char_g15= np.array([3,-1,1,-1,0,-3,1,-1,1,0])
char_g25= np.array([3,-1,-1,1,0,-3,1,1,-1,0])

char_g1p= np.array([1, 1, 1, 1, 1,-1,-1,-1,-1,-1])
char_g2p= np.array([1, 1,-1,-1, 1,-1,-1, 1, 1,-1])
char_g12p=np.array([2, 2, 0, 0,-1,-2,-2, 0, 0, 1])
char_g15p=np.array([3,-1, 1,-1, 0, 3,-1, 1,-1, 0])
char_g25p=np.array([3,-1,-1, 1, 0, 3,-1,-1, 1, 0])

Oh_sym = [[char_g1,"$\Gamma_1$"],[char_g2,"$\Gamma_2$"],
          [char_g12,"$\Gamma_{12}$"],
          [char_g15,"$\Gamma_{15}$"],
          [char_g25,"$\Gamma_{25}$"]]
Oh_symp = [[char_g1p,"$\Gamma^'_1$"],[char_g2p,"$\Gamma^'_2$"],
          [char_g12p,"$\Gamma^'_{12}$"],
          [char_g15p,"$\Gamma^'_{15}$"],
          [char_g25p,"$\Gamma^'_{25}$"]]

Nk = [1,3,6,6,8,1,3,6,6,8]
'''
for char_g in Oh_sym:
    if (char_coef(char_g[0],char_G,Nk)/48)==1:
          print(char_g[1])
for char_g in Oh_symp:
    if (char_coef(char_g[0],char_G,Nk)/48)==1:
          print(char_g[1])
'''

'\nfor char_g in Oh_sym:\n    if (char_coef(char_g[0],char_G,Nk)/48)==1:\n          print(char_g[1])\nfor char_g in Oh_symp:\n    if (char_coef(char_g[0],char_G,Nk)/48)==1:\n          print(char_g[1])\n'

In [251]:
ir_rep=[]
for i in range(len(Deg_char)):
    for char_g in Oh_sym:
        c=char_coef(char_g[0],Deg_char[i],Nk)/48
        if (c)==1:
              ir_rep.append(char_g[1])
    for char_g in Oh_symp:
        c=char_coef(char_g[0],Deg_char[i],Nk)/48
        if (c)==1:
              ir_rep.append(char_g[1])

In [256]:
for i in range(len(ir_rep)):
    print(deg_u[i],ir_rep[i],Deg_char[i])

[0, 1] $\Gamma^'_{12}$ [ 2  2  0  0 -1 -2 -2  0  0  1]
[2, 3, 4] $\Gamma_{15}$ [ 3 -1  1 -1  0 -3  1 -1  1  0]
[5, 6, 7] $\Gamma^'_{25}$ [ 3 -1 -1  1  0  3 -1 -1  1  0]
[8] $Gamma^'_2$ [ 1  1 -1 -1  1 -1 -1  1  1 -1]
[9, 10, 11] $\Gamma_{15}$ [ 3 -1  1 -1  0 -3  1 -1  1  0]
[12, 13] $\Gamma^'_{12}$ [ 2  2  0  0 -1 -2 -2  0  0  1]
[14, 15, 16] $\Gamma_{25}$ [ 3 -1 -1  1  0 -3  1  1 -1  0]
[17] $Gamma^'_2$ [ 1  1 -1 -1  1 -1 -1  1  1 -1]
[18, 19, 20] $\Gamma^'_{25}$ [ 3 -1 -1  1  0  3 -1 -1  1  0]
[21] $\Gamma_1$ [1 1 1 1 1 1 1 1 1 1]
[22, 23, 24] $\Gamma^'_{25}$ [ 3 -1 -1  1  0  3 -1 -1  1  0]
[25, 26] $\Gamma_{12}$ [ 2  2  0  0 -1  2  2  0  0 -1]
[27, 28, 29] $\Gamma_{15}$ [ 3 -1  1 -1  0 -3  1 -1  1  0]
[30, 31, 32] $\Gamma^'_{15}$ [ 3 -1  1 -1  0  3 -1  1 -1  0]
[33] $Gamma^'_2$ [ 1  1 -1 -1  1 -1 -1  1  1 -1]
[34, 35, 36] $\Gamma_{15}$ [ 3 -1  1 -1  0 -3  1 -1  1  0]
[37, 38, 39] $\Gamma^'_{25}$ [ 3 -1 -1  1  0  3 -1 -1  1  0]
[40, 41, 42] $\Gamma^'_{15}$ [ 3 -1  1 -1  0  3 -1  1 -1

Using this, we can also see how a product of two or more irrep decomposes (i.e. what irreps a product of who irreps contains): 

In [203]:
char_red = char_g15*char_g25p
print(char_red)

[ 9  1 -1 -1  0 -9 -1  1  1  0]


In [204]:
for char_g in Oh_sym:
    c=char_coef(char_g[0],char_red,Nk)/48
    if (c)!=0:
          print(char_g[1],c)
for char_g in Oh_symp:
    c=char_coef(char_g[0],char_red,Nk)/48
    if (c)!=0:
          print(char_g[1],c)

$\Gamma_{15}$ 1.0
$\Gamma_{25}$ 1.0
$Gamma^'_2$ 1.0
$\Gamma^'_{12}$ 1.0


Using a simmilar approach for charachters, we now wich to get full matrices of our representations and see if there is a unitary transformation that can transform them to a bassis that corresponds one given in literature (e.g. to the form shown in tables on Bilbao Crystallographic Server).

For that we define function `Gather_Matrices` that takes in start and stop indices of degenerate eigenvalues (eigenfunctions), list of symmetries obtained from fucntion `Gather_Symmetries` and our lists of k-points and wave-functions obtained from `Gather_G` and `Gather_C`. To see details, set `details=True`.

In [8]:
def Gather_Matrices(k_start,k_stop,symmetry_list,G_R,C_R,k_Rpoint,details=False):
    Matrices=[]
    dim_G = 2*abs(max(np.amax(G_R),np.amin(G_R),key=abs))+1
    if details: print('Dimension of 3x3 square matrix is:', dim_G)
    M=np.zeros((dim_G,dim_G,dim_G),dtype=int)
    if details: print('Creating G-matrix')
    for i in range(len(G_R)):
        n1,n2,n3 = G_R[i,0],G_R[i,1],G_R[i,2]
    #    print(n1,n2,n3, i)
        M[n1,n2,n3] = i
    if details: print('G-matrix: Done')

    for s in range(len(symmetry_list)):
        sym_name=symmetry_list[s].operation
        sym_matrix=symmetry_list[s].matrix
        if details: print('\nNow performing for '+ sym_name)
        dim = k_stop-k_start+1
        full_mat=np.zeros((dim,dim),dtype=complex)
        for m in range(k_start,k_stop+1):
            RC_R = np.zeros(len(G_R),dtype=complex)
            for i in range(len(G_R)):
                g_i = np.matmul(sym_matrix,G_R[i]+k_Rpoint)-k_Rpoint
                n1,n2,n3=int(g_i[0]),int(g_i[1]),int(g_i[2])
                index = M[n1,n2,n3]
                RC_R[i] = C_R[m,index]
            for n in range(k_start,k_stop+1):
                mat_comp=np.vdot(RC_R,C_R[n])
                full_mat[m-k_start,n-k_start]= (mat_comp)
        if details: print('Matrix = \n',full_mat,'\nSym. Matrix =\n',sym_matrix)
        Matrices.append(Symmetry(full_mat,sym_name))
        #Matrices.append([full_mat, sym_name,np.trace(full_mat)])
    if details: print('Done')
    return np.array(Matrices)

In [295]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_R=31
#E=Gather_E(path,units='Hartree')
#E_G = E[k_G-1]
k_points= Gather_K(path)
k_Rpoint = k_points[k_R-1]
G_R = Gather_G(path,k_R)
C_R = Gather_C(path,k_R)
symmetry_list = Gather_Symmetries(path)

Here is an example for a 3-times degenerate eigenvalue:

In [515]:
k_start=18
k_stop=20
matrices=Gather_Matrices(k_start,k_stop,symmetry_list,G_R,C_R,details=True)

Dimension of 3x3 square matrix is: 31
Creating G-matrix
G-matrix: Done

Now performing for identity
Matrix = 
 [[ 1.00000000e+00+0.00000000e+00j -3.47549212e-16-3.06728716e-16j
  -2.54965599e-16-1.10416198e-15j]
 [-3.47549212e-16+3.06728716e-16j  1.00000000e+00+0.00000000e+00j
   1.77543961e-15-1.98015127e-17j]
 [-2.54965599e-16+1.10416198e-15j  1.77543961e-15+1.98015127e-17j
   1.00000000e+00+0.00000000e+00j]] 
Sym. Matrix =
 [[1 0 0]
 [0 1 0]
 [0 0 1]]

Now performing for 180 deg rotation - cart. axis [0,0,1]
Matrix = 
 [[ 0.3711844 -7.98038444e-17j -0.44712383-4.92669864e-01j
   0.39988951-5.09575514e-01j]
 [-0.44712383+4.92669864e-01j -0.67718177-1.53456959e-16j
   0.0526934 +3.09846632e-01j]
 [ 0.39988951+5.09575514e-01j  0.0526934 -3.09846632e-01j
  -0.69400262-2.55500077e-17j]] 
Sym. Matrix =
 [[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]

Now performing for 180 deg rotation - cart. axis [0,1,0]
Matrix = 
 [[-0.96960759-6.76172320e-16j -0.13076323-4.64155955e-02j
  -0.06419978+1.91013315

Matrix = 
 [[ 4.79329969e-01-0.0497774j  -7.74003831e-04-0.62942839j
  -5.44964938e-01-0.27312545j]
 [ 3.15155501e-01-0.20486404j -4.28754704e-01+0.51727376j
   1.87910538e-02-0.63792782j]
 [-7.76481329e-01-0.15345838j -2.04955010e-01-0.33228002j
  -5.05752653e-02-0.46749636j]] 
Sym. Matrix =
 [[ 0  0 -1]
 [ 1  0  0]
 [ 0 -1  0]]

Now performing for inversion
Matrix = 
 [[ 1.00000000e+00+1.13940649e-16j -5.20257152e-16-3.93246166e-16j
  -4.23340900e-16-9.30029312e-16j]
 [-3.55362350e-16+3.63679876e-16j  1.00000000e+00-1.54551484e-16j
   2.23140414e-15+1.06709454e-15j]
 [-4.54143014e-16+7.96751553e-16j  2.33138841e-15+1.41255767e-16j
   1.00000000e+00-6.99016534e-16j]] 
Sym. Matrix =
 [[-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]

Now performing for inv. 180 deg rotation - cart. axis [0,0,1]
Matrix = 
 [[ 0.3711844 -3.69816014e-16j -0.44712383-4.92669864e-01j
   0.39988951-5.09575514e-01j]
 [-0.44712383+4.92669864e-01j -0.67718177-9.98886632e-17j
   0.0526934 +3.09846632e-01j]
 [ 0.39988951+5.09

Matrix = 
 [[-0.45262944+0.25216486j  0.19498322-0.28212014j -0.78342144+0.01342006j]
 [-0.39658047+0.14846192j  0.69294741-0.15025084j  0.50768972+0.24532164j]
 [ 0.70905463+0.22240502j  0.60035408+0.13861209j -0.24031798-0.10191402j]] 
Sym. Matrix =
 [[ 0  0 -1]
 [ 1  0  0]
 [ 0  1  0]]

Now performing for inv. 120 deg rotation - cart. axis [1,-1,-1]
Matrix = 
 [[ 0.40961856-0.12434628j -0.35166259+0.36680631j  0.73950627-0.10801704j]
 [ 0.3864288 -0.76408746j -0.14816862-0.45339331j -0.17427543+0.09461378j]
 [ 0.18512705-0.22208501j  0.26285363+0.66721442j -0.26144994+0.57773959j]] 
Sym. Matrix =
 [[ 0  0  1]
 [ 1  0  0]
 [ 0 -1  0]]

Now performing for inv. 120 deg rotation - cart. axis [-1,-1,1]
Matrix = 
 [[ 4.79329969e-01-0.0497774j  -7.74003831e-04-0.62942839j
  -5.44964938e-01-0.27312545j]
 [ 3.15155501e-01-0.20486404j -4.28754704e-01+0.51727376j
   1.87910538e-02-0.63792782j]
 [-7.76481329e-01-0.15345838j -2.04955010e-01-0.33228002j
  -5.05752653e-02-0.46749636j]] 
Sym. Matri

We will now use function `Gather_Symmetries` to read out matrices of representation that we obtained from Bilbao Crystallographic Server.

In [495]:
path2="/home/mjocic/data_paradox"
G_25p = Gather_Symmetries(path2,"G25p_t2g.xml")

### Explicit constructions of unitary transformations between equivalent irreducible representations 

We refer the reader to full paper by M. Mozrzymas , M. Studziński and M.Horodecki that can be foud on arxiv (https://arxiv.org/pdf/1405.2169.pdf). Here we will present just the functions `Compute_small_r` and `Gather_Unitary_Transforms`.

As there are more than one unitary transformations that can relate who equivalent irreps, we use function `Compute_small_r` to get a matrix $r:= r_{ab}$ that tells us how many unitary transformations are available and determine the weights that norm them (the terms $r_{ab}$ in matrix give a weighing factor for indices $ab$, if $r_{ab}=0$ then no unitary transformation corresponds for indices $ab$).

In [612]:
import numpy.linalg as nplin
def Compute_small_r(sym1,sym2):
    if len(sym1)!=len(sym2):
        raise ValueError("Irreps are not compatible")
    order=len(sym1)
    dim = int(round(sym1[0].char))
    inv = int(round(order/2))
    r = np.zeros((dim,dim),dtype=complex)
    sq = np.sqrt(dim/order)
    for i in range(dim):
        for j in range(dim):
            s = 0+0j
            for g in range(order):
                s1 = 0+0j
                s1 = sym1[g].matrix[i,i]*nplin.inv(sym2[g].matrix)[j,j]
                s += s1
            r[i,j] = np.sqrt(s)
    return sq*r
    #else: print("Error: Representations are not of the same order!")

Here, we compute matrix $r_{ab}$: 

In [614]:
r = Compute_small_r(matrices,G_25p)
print(r.real)

[[0.82800495 0.54700237 0.1232729 ]
 [0.40175753 0.72238741 0.56280308]
 [0.39115047 0.42301871 0.81734722]]


/home/mjocic/user_install/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.


We can use a formula from refered papaer as a check if we're on the right path:

In [615]:
s=0
for i in range(len(r)):
    s += r[0,i]*r[0,i]
print(s)

(0.9999999999999932+4.336808689942018e-17j)


Also we define a function `Gather_Unitary_Transforms` that takes as input matrix $r$ and matrices of two irreps that we want to connect with unitary transformation. Our function returns a block matrix that contains unitary matrices of transformation that correspond to indices $ab$ from matrix $r_{ab}$:

In [650]:
import numpy.linalg as nplin
def Gather_Unitary_Transforms(r,sym1,sym2):
    if len(sym1)!=len(sym2):
        raise ValueError("Irreps are not compatible")
    dim = len(r)
    order=len(sym1)
    inv = int(round(order/2))
    great_u = []
    for m in range(dim):
        g_u=[]
        for n in range(dim):
            g_u.append(0)
        great_u.append(g_u)
    factor=dim/order
    for a in range(dim):
        for b in range(dim):
            if abs(r[a,b])>1e-14:
                small_u = np.zeros((dim,dim),dtype=complex)
                for i in range(dim):
                    for j in range(dim):
                        u_ij=0+0j
                        for g in range(order):
                            u1 = nplin.inv(sym1[g].matrix)[i,a]*sym2[g].matrix[b,j]
                            #u2 = sym1[g+inv].matrix[i,i]*sym2[g].matrix[j,j]
                            u_ij += u1 #+u2
                        small_u[i,j]=1/r[a,b] *u_ij
                great_u[a][b]=np.array(small_u)
            else: 
                great_u[a][b]=0
    #great_u=np.array(great_u)
    return np.array(great_u)

In [660]:
Unitary_transforms = Gather_Unitary_Transforms(r,matrices,G_25p)

In [661]:
print(Unitary_transforms)

[[[[ 13.24807922+2.01126157e-16j  -8.5888233 +1.68234429e+00j
     -0.25797461+1.95542274e+00j]
   [ -4.32001117+4.76006684e+00j  -6.77855174+9.36179426e+00j
     -1.8764063 -8.80717952e+00j]
   [  3.86364369+4.92340547e+00j   5.71353379+3.62841677e+00j
     12.83457423-2.50921578e+00j]]

  [[-13.00101901-2.54658750e+00j   8.75203793+4.82041729e-16j
      0.62904122-1.86936784e+00j]
   [  5.15444337-3.84089136e+00j   8.45169374-7.88421387e+00j
      0.14846987+9.00362529e+00j]
   [ -2.84519904-5.57427183e+00j  -4.90951779-4.65902476e+00j
    -13.07755472-4.68025167e-03j]]

  [[ -1.73277548-1.31342717e+01j   2.7912627 +8.29499973e+00j
      1.97236634-4.72826635e-15j]
   [  5.28420908+3.66030973e+00j  10.16796889+5.49584972e+00j
     -8.48609792+3.01221741e+00j]
   [  4.37576776-4.47440728e+00j   2.84994826-6.13902855e+00j
     -4.16635169-1.23961273e+01j]]]


 [[[ -8.90335682-9.81029258e+00j   7.01789645+5.22946435e+00j
      1.62137536-1.12310772e+00j]
   [  6.42812048+1.65804936e-15j

We can now use one unitary matix in block and generate an iverse to use for our transformation:

In [631]:
import numpy.linalg as nplin

u_11 = Unitary_transforms[0,0]
u_11i= nplin.inv(u_11)
print(u_11)
print(u_11i)
print(np.matmul(u_11,u_11i).real)

[[13.24807922+2.01126157e-16j -8.5888233 +1.68234429e+00j
  -0.25797461+1.95542274e+00j]
 [-4.32001117+4.76006684e+00j -6.77855174+9.36179426e+00j
  -1.8764063 -8.80717952e+00j]
 [ 3.86364369+4.92340547e+00j  5.71353379+3.62841677e+00j
  12.83457423-2.50921578e+00j]]
[[ 0.05175031+2.53847116e-17j -0.01687504-1.85940111e-02j
   0.01509236-1.92320526e-02j]
 [-0.03355009-6.57165738e-03j -0.02647872-3.65695088e-02j
   0.02231849-1.41735030e-02j]
 [-0.00100771-7.63837007e-03j -0.00732971+3.44030450e-02j
   0.05013506+9.80162412e-03j]]
[[ 1.00000000e+00 -5.63785130e-17 -2.34187669e-17]
 [ 2.60208521e-18  1.00000000e+00  5.55111512e-17]
 [ 4.94396191e-17 -5.55111512e-17  1.00000000e+00]]


And we check if our unitary transformation works:

In [125]:
for i in range(len(matrices)):
    A = np.matmul(u_11i,matrices[i].matrix)
    B = np.matmul(A,u_11).real
    print(np.rint(B),'\n',G_25p[i].matrix,'\n****\n')
    print(np.rint(B) == G_25p[i].matrix)

[[ 1.  0. -0.]
 [ 0.  1. -0.]
 [-0. -0.  1.]] 
 [[1 0 0]
 [0 1 0]
 [0 0 1]] 
****

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
[[ 1. -0.  0.]
 [ 0. -1.  0.]
 [-0.  0. -1.]] 
 [[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]] 
****

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
[[-1. -0. -0.]
 [-0. -1. -0.]
 [ 0.  0.  1.]] 
 [[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]] 
****

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
[[-1.  0.  0.]
 [-0.  1.  0.]
 [ 0.  0. -1.]] 
 [[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]] 
****

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
[[ 1.  0. -0.]
 [ 0.  0. -1.]
 [-0. -1.  0.]] 
 [[ 1  0  0]
 [ 0  0 -1]
 [ 0 -1  0]] 
****

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
[[ 1. -0.  0.]
 [ 0. -0.  1.]
 [-0.  1.  0.]] 
 [[1 0 0]
 [0 0 1]
 [0 1 0]] 
****

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
[[-1. -0. -0.]
 [-0. -0. -1.]
 [ 0.  1.  0.]] 
 [[-1  0  0]
 [ 0  0 -1]
 [ 0  1  0

## Using Unitary Transforms ... work in progress

In [4]:
from post_procesing_tools import *

path = "/home/mjocic/data_paradox/nbnd_120"
k_R=31
k_points= Gather_K(path)
k_Rpoint = k_points[k_R-1]

G_R = Gather_G(path,k_R)
C_R = Gather_C(path,k_R)
symmetry_list = Gather_Symmetries(path)

k_start=18
k_stop=20
matrices=Gather_Matrices(k_start,k_stop,symmetry_list,G_R,C_R,k_Rpoint)

path2="/home/mjocic/data_paradox"
G_25p = Gather_Symmetries(path2,"G25p_t2g.xml")

r = Compute_small_r(matrices,G_25p)
Unitary_transforms = Gather_Unitary_Transforms(r,matrices,G_25p)

u_11 = Unitary_transforms[0,0]
u_11i= nplin.inv(u_11)

/home/mjocic/Post_processing/CsPbBr3/CsPbBr3_collect_wf/post_procesing_tools.py:377: ComplexWarning: Casting complex values to real discards the imaginary part
  dim = int(round(sym1[0].char))


In [27]:
u_11a = np.transpose(np.conjugate(u_11))
norm = np.trace(u_11a.dot(u_11))/len(u_11)
u_11 = u_11/np.sqrt(norm)
u_11a= u_11a/np.sqrt(norm)

In [33]:
basis_qe=C_R[18:21:1]
basis_g25p = u_11.T.dot(basis_qe)

(3, 15192)

In [36]:
matrices_new = Gather_Matrices(0,2,symmetry_list,G_R,basis_g25p,k_Rpoint)

In [37]:
for i in range(len(matrices_new)):
    print(matrices[i].operation)
    #print(abs(matrices_new[i].matrix-matrices[i].matrix)<1e-10)
    #print(matrices[i].matrix)
    to_print = matrices_new[i].matrix
    print(np.rint(to_print).real)
    print(G_25p[i].matrix)
    print(G_25p[i].matrix == np.rint(to_print).real)

identity
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
180 deg rotation - cart. axis [0,0,1]
[[ 1.  0. -0.]
 [ 0. -1. -0.]
 [-0. -0. -1.]]
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
180 deg rotation - cart. axis [0,1,0]
[[-1. -0.  0.]
 [-0. -1. -0.]
 [ 0. -0.  1.]]
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
180 deg rotation - cart. axis [1,0,0]
[[-1. -0. -0.]
 [-0.  1.  0.]
 [-0.  0. -1.]]
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
180 deg rotation - cart. axis [1,1,0]
[[ 1. -0. -0.]
 [-0.  0. -1.]
 [-0. -1. -0.]]
[[ 1  0  0]
 [ 0  0 -1]
 [ 0 -1  0]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
180 deg rotation - cart. axis [1,-1,0]
[[ 1.  0. -0.]
 [ 0. -0.  1.]
 [-0.  1.  0.]]
[[1 0 0]
 [0 0 1]
 [0 1 0]]
[[ True  

In [95]:
kdp = []
kkR = np.dot(k_Rpoint,k_Rpoint)
basis4 = [C_R[17],basis_g25p[1],basis_g25p[2],basis_g25p[0]]
basis4 = np.array(basis4)
print(basis4.shape)
for m in range(len(basis4)):
    for n in range(len(basis4)):
        CGC = Compute_Matrix_Element(G_R,basis4,m,n)
        if (abs(CGC)>1e-10).any:
                print('kdp[{},{}]=\n{}\n'.format(m,n,CGC))
                kdp.append([CGC,[m,n]])

(4, 15192)
kdp[0,0]=
[-0.5+1.06706299e-22j -0.5+1.65436123e-24j -0.5+5.29395592e-23j]

kdp[0,1]=
[-4.04775991e-01+4.66634739e-01j  2.53606342e-10+2.33358129e-10j
  1.67378738e-10+1.60664480e-10j]

kdp[0,2]=
[ 7.62305094e-11-2.82586079e-10j -4.04775991e-01+4.66634739e-01j
 -3.55487969e-10-1.55858761e-10j]

kdp[0,3]=
[ 1.82769449e-10+2.83320910e-12j  1.93887805e-10+1.61457010e-10j
 -4.04775990e-01+4.66634738e-01j]

kdp[1,0]=
[-4.04775991e-01-4.66634739e-01j  2.53606342e-10-2.33358129e-10j
  1.67378738e-10-1.60664480e-10j]

kdp[1,1]=
[-0.5+1.35525272e-20j -0.5+6.77626358e-21j -0.5+0.00000000e+00j]

kdp[1,2]=
[2.51977703e-10-2.88713720e-10j 9.85028835e-12+8.56783655e-11j
 1.89668568e-10-1.39355521e-10j]

kdp[1,3]=
[ 9.12304334e-11+2.80536569e-10j -4.21708247e-11-5.51117675e-11j
  8.51410078e-11+1.91554267e-10j]

kdp[2,0]=
[ 7.62305096e-11+2.82586079e-10j -4.04775991e-01-4.66634739e-01j
 -3.55487969e-10+1.55858768e-10j]

kdp[2,1]=
[2.51977703e-10+2.88713720e-10j 9.85028835e-12-8.56783655e-1

In [43]:
kdp = []
kkR = np.dot(k_Rpoint,k_Rpoint)
basis4 = [C_R[17],basis_qe[0],basis_qe[1],basis_qe[2]]
basis4 = np.array(basis4)
print(basis4.shape)
for m in range(len(basis4)):
    for n in range(len(basis4)):
        CGC = Compute_Matrix_Element(G_R,basis4,m,n)
        if (abs(CGC)>1e-10).any:
                print('kdp[{},{}]=\n{}\n'.format(m,n,CGC))
                kdp.append([CGC,[m,n]])

(4, 15192)
kdp[0,0]=
[-0.5+1.06706299e-22j -0.5+1.65436123e-24j -0.5+5.29395592e-23j]

kdp[0,1]=
[ 0.26634936-0.20792942j  0.06355563+0.04194552j -0.33515652+0.38637587j]

kdp[0,2]=
[ 0.44452084+0.03914511j -0.20938823-0.2775332j   0.24811558-0.00556916j]

kdp[0,3]=
[-0.03872225+0.25842683j -0.39787592+0.31083674j  0.04584511+0.23723667j]

kdp[1,0]=
[ 0.26634936+0.20792942j  0.06355563-0.04194552j -0.33515652-0.38637587j]

kdp[1,1]=
[-0.5-2.11758237e-22j -0.5+0.00000000e+00j -0.5+0.00000000e+00j]

kdp[1,2]=
[-2.83284228e-10-1.04949132e-10j  1.62101258e-11-6.83211838e-11j
 -4.84856465e-11+1.07390834e-10j]

kdp[1,3]=
[-2.43828855e-10-1.30975558e-10j  1.67014295e-10-1.63341444e-10j
 -1.81002091e-10+2.57708426e-11j]

kdp[2,0]=
[ 0.44452084-0.03914511j -0.20938823+0.2775332j   0.24811558+0.00556916j]

kdp[2,1]=
[-2.83284228e-10+1.04949133e-10j  1.62101258e-11+6.83211838e-11j
 -4.84856465e-11-1.07390834e-10j]

kdp[2,2]=
[-0.5+0.00000000e+00j -0.5+0.00000000e+00j -0.5-6.77626358e-21j]

kdp[2,

In [245]:
# k_points - k-mesh
# k_Rpoint - Rpoint
# G_R - G_R mesh
# E_R - E_R mesh
# basis3 - 4*4 basis
# a = lattice constant
# units = Hartree, Rydberg, Electronvol
path = "/home/mjocic/data_paradox/nbnd_120"
E = Gather_E(path,units='Hartree')
E_R = E[30][17:20+1:1]
print(E_R)
Ham44kdp = Compute_H(k_points,k_Rpoint,E_R,G_R,basis3,a=10.8,units='Electronvolt')

[0.11328372 0.15077892 0.15077892 0.15077892]
Calculating bands: DONE


In [225]:
# dft results
basis3_dft =np.array([C_R[17],C_R[18],C_R[19],C_R[20]])
Ham33dft = Compute_H(k_points,k_Rpoint,E_R,G_R,basis3_dft,a=10.8,units='Hartree')

Calculating bands: DONE


In [249]:
# Prepare results for ploting
path = "/home/mjocic/data_paradox/nbnd_120"
E = Gather_E(path,units='Electronvolt')
path = '/home/mjocic/Desktop/'
k2=[0.0]
d=0
for i in range(len(k_points)-1):
    d1 = k_points[i+1]-k_points[i]
    d2 = np.sqrt(np.vdot(d1,d1))
    d += d2
    k2.append(d)
k2=np.array(k2)
x=[];y_kp=[];y_dft=[]
FILE_kp = open( path+'/plotband_kp.dat','w')
FILE_dft = open( path+'/plotband_dft.dat', 'w')
for n in range(Ham44kdp.shape[1]):
    for k_i in range(len(k2)):
        x.append(k2[k_i])
        y_kp.append(Ham44kdp[k_i][n].real);
        y_dft.append(E[k_i][n+17])
        FILE_kp.write("{} {}\n".format (k2[k_i],Ham44kdp[k_i][n].real))
        FILE_dft.write("{} {}\n".format (k2[k_i],E[k_i][n+17].real))
    FILE_kp.write("\n")
    FILE_dft.write("\n")
FILE_kp.close()
FILE_dft.close()

In [251]:
# Plot the results
import matplotlib.pyplot as plt
%matplotlib
plt.plot(x,y_kp,label='$k \cdot p$')
plt.plot(x,y_dft, label='dft')
plt.xlim(k2[0],k2[-1])
#plt.ylim(-0.1,0.3)
plt.xticks([k2[0],k2[10],k2[20],k2[30],k2[40]],('$\Gamma$','M','X','R','M'))
plt.ylabel('E(k) [eV]')
plt.legend()
plt.show()

Using matplotlib backend: Qt5Agg


In [44]:
char_red = char_g25p*char_g15*char_g2p
print(char_red)

for char_g in Oh_sym:
    c=char_coef(char_g[0],char_red,Nk)/48
    if (c)!=0:
          print(char_g[1],c)
for char_g in Oh_symp:
    c=char_coef(char_g[0],char_red,Nk)/48
    if (c)!=0:
          print(char_g[1],c)

[9 1 1 1 0 9 1 1 1 0]
$\Gamma_1$ 1.0
$\Gamma_{12}$ 1.0
$\Gamma^'_{15}$ 1.0
$\Gamma^'_{25}$ 1.0


In [48]:
dim_G = 2*abs(max(np.amax(G_R),np.amin(G_R),key=abs))+1
#print(dim_G)
M=np.zeros((dim_G,dim_G,dim_G),dtype=int)
#print('Creating G-matrix')
for i in range(len(G_R)):
    n1,n2,n3 = G_R[i,0],G_R[i,1],G_R[i,2]
    #    print(n1,n2,n3, i)
    M[n1,n2,n3] = i

31
Creating G-matrix


In [85]:
print(G_R[511])
print(M[4,0,-3])

[ 4  0 -3]
511


In [88]:
print(M[-4,0,3])
print(k_points[30])

488
[0.5 0.5 0.5]


In [89]:
print(G_R[488])

[-4  0  3]


In [116]:
print(C_R[18,511],C_R[18,488])
print(path)

(-0.001020475303105298-0.001631646022886768j) (-0.004068567605534394-0.005325765994984152j)
/home/mjocic/data_paradox/nbnd_120


In [106]:
A = basis4[3]*G_R[:,0]
print(G_R[:,0].shape,basis4[3].shape)
print(np.vdot(basis4[3],A))

(15192,) (15192,)
(-0.49999999989399957+0j)


In [114]:
off_diag = []
for m in range(len(basis4)):
    for n in range(len(basis4)):
        off_diag.append([Compute_Matrix_Element(G_R,basis4,m,n),[m,n]])

In [117]:
G_0 = Gather_G(path,1)
C_0 = Gather_C(path,1)

In [120]:
A = C_0[17]*G_0[:,0]
B = C_0[17]
print(np.vdot(B,A))

(2.008169828959307e-14-2.0514079191315086e-22j)


In [67]:
from post_procesing_tools import *
path = "/home/mjocic/data_paradox/nbnd_120"
point = 30
G_X = Gather_G(path,point+1)
C_X = Gather_C(path,point+1)
k_points = Gather_K(path)
K_X = k_points[point]
print(K_X)

[0.5 0.5 0.5]


In [68]:
KPx=[]
for band in range(len(C_X)):
    Ax,Ay,Az = C_X[band]*G_X[:,0], C_X[band]*G_X[:,1], C_X[band]*G_X[:,2]
    B = C_X[band]
    kpx = np.array([np.vdot(B,Ax),np.vdot(B,Ay),np.vdot(B,Az)])
    #print("band={}\npx={}\npy={}\npz={}".format(band,kpx[0],kpx[1],kpx[2]))
    #print(k_points[point])
    #print(k_points[point]+kpx)
    KPx.append(kpx)
KPx = np.array(KPx)

In [69]:
import matplotlib.pyplot as plt
%matplotlib
x = np.arange(len(KPx))
plt.plot(x,KPx[:,0].real,'o-',label='px',color='blue')
plt.plot(x,KPx[:,1].real,'x',label='py', color='red')
plt.plot(x,KPx[:,2].real,'k',label='pz', color='green')
#plt.axhline(y=-K_X[0])
#plt.axhline(y=-K_X[1])
#plt.axhline(y=-K_X[2])
plt.legend()
plt.grid()
plt.show()

Using matplotlib backend: Qt5Agg
